## A simple math problem

There is a group of about 300 people who want to help each other. Unfortunatly a single person can meaningfully keep track of 150 people. Hovewer, all 300 want to cooperate and do it fairly to each other. How can they do that?

It's sad that you will not find the problem in any school book. If you did, the initial answer would be simple. Why don't they write down who helped whom and how much.

In [1]:
from dataclasses import dataclass, field
from time import time

@dataclass
class Transaction:
    recipient: str
    sender: str
    amount: float
    timestamp: float = field(default_factory=time)

## Key block

A block of transactions can help people keep track of provided help so you don't need to remember how much help you got and how much you recived.

In [2]:
block = []
block.append(Transaction(
    sender='Alise', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [3]:
block

[Transaction(recipient='Bob', sender='Alise', amount=2.0, timestamp=1542722885.5342875),
 Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1542722885.534319)]

This covers direct person to person cooperation. Everyone is happy but if someone tries to cooperate inderectly, things fall apart. Since anyone can update the block, transactions can be removed. If people try to exchange units of help there is an insentive to mess with the transaction list.

In order to prevent temparing with transactions in a block we can hash the block and include the hash in the next block. If someone changes transactions the hash will be different and the temparing will be apparant.

In [4]:
import hashlib

class Block:
    transactions: list
    previous_hash: str = 'coinbase'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}'.format(self.transactions.__repr__(), self.previous_hash)
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [5]:
block = Block()
block.append(Transaction(
    sender='Alise', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [6]:
block

transactions: [Transaction(recipient='Bob', sender='Alise', amount=2.0, timestamp=1542722885.867171), Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1542722885.8672497)]

previous hash: coinbase

In [7]:
block.hash()

'feec7bb392b8b4af3aa03a6eb62b9a07e275b659d460b81d724fb833b982ab39'

Hey, look at that! We got short and unique (for our purpuses) representation of our block and all the transactions in it.

In [8]:
block.append(Transaction(
    sender='Jon', recipient='Bob', amount=1.0
))

In [9]:
block.hash()

'46946d5e791572fbb7e7ebb849fa2d029cf32aea5e6945f27506601507665a55'

## Verification chain

Now anyone can put the blocks into a chain and it's easy to prove that no one tampared with prevous transactions.

In [10]:
import itertools


class Chain:
    blocks: list

    def __init__(self):
        """We need initial block"""
        block = Block()
        block.append(Transaction(
            sender='Alise', recipient='Alise', amount=2.0
        ))
        self.blocks = [block]

    def push(self, block: Block):
        block.previous_hash = self.blocks[-1].hash()
        self.blocks.append(block)

    @property
    def tempared(self):
        """
        It gives you possition of invalid block according to it's naighbour to the right.
        If chain looks good it returns 0.
        """
        a, b = itertools.tee(self.blocks)
        next(b, None)
        for position, pair in enumerate(zip(a, b)):
            if pair[0].hash() != pair[1].previous_hash:
                return position
        return -1

    @property
    def is_valid(self):
        if self.tempared < 0:
            return True
        return False
        

In [11]:
chain = Chain()

In [12]:
chain.blocks

[transactions: [Transaction(recipient='Alise', sender='Alise', amount=2.0, timestamp=1542722886.5601923)]
 
 previous hash: coinbase]

In [13]:
for _ in range(10):
    block = Block()
    for j in range(3):
        block.append(Transaction(sender='Bob', recipient='Jon', amount=j))
    chain.push(block)

In [14]:
chain.is_valid

True

In [15]:
borrowed = chain.blocks[0].transactions.pop()
borrowed

Transaction(recipient='Alise', sender='Alise', amount=2.0, timestamp=1542722886.5601923)

In [16]:
chain.is_valid

False

In [17]:
chain.tempared

0

Let's put borrowed transaction back.

In [18]:
chain.blocks[0].transactions.append(borrowed)

In [19]:
chain.is_valid

True

We can only mess with transactions in the last block.

In [20]:
chain.blocks[-1].append(Transaction(recipient='Alise', sender='Jon', amount=5))

In [21]:
chain.is_valid

True

## Equal opportunity to verify

Now who should have a right to create blocks and add them to the chain? Everyone who wants to do it! We just need to make sure it's not monopolized by a single person. Somehow we need to give equal-ish chance for everyone to verify previous blocks and add new ones. Here comes the elegant part. What if we allow previous hash to only start with `00`. Our block will have a special `nonce` field that can have anything that makes our block hash to start with `00`. If we use good hashing algorithm there is no other way but to try different possibilities untill we get what we need. This kind of help work should be rewarded. We'll allow that person to assign help units to herself out of nothing.

In [22]:
class Block:
    transactions: list
    previous_hash: str = 'coinbase'
    nonce: str = 'blalba'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}\n\nnonce: {}'.format(
            self.transactions.__repr__(), self.previous_hash, self.nonce
        )
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [23]:
from uuid import uuid4

def add_block_with_proof_of_work(chain: Chain, block: Block):
    last_block = chain.blocks[-1]
    for i in range(100000):
        last_block.nonce = uuid4()
        temp_hash = last_block.hash()
        if temp_hash.startswith('00'):
            block.previous_hash = temp_hash
            chain.blocks.append(block)
            print('Found hash {} after {} iterations'.format(temp_hash, i))
            break
    else:
        print('After {} iterations needed hash was not found.'.format(i))    

In [24]:
chain = Chain()

In [25]:
block = Block()
for j in range(3):
    block.append(Transaction(sender='Bob', recipient='Jon', amount=j))

In [26]:
add_block_with_proof_of_work(chain, block)

Found hash 00b718417a69733cd1b8a5307cce3b2c8fe3115e89f1d52e436e7ec9a1ab607c after 15 iterations


In [27]:
chain.blocks

[transactions: [Transaction(recipient='Alise', sender='Alise', amount=2.0, timestamp=1542722887.949511)]
 
 previous hash: coinbase
 
 nonce: 3cc29415-3661-4450-b593-e1d8fcf96123,
 transactions: [Transaction(recipient='Jon', sender='Bob', amount=0, timestamp=1542722888.0491416), Transaction(recipient='Jon', sender='Bob', amount=1, timestamp=1542722888.0491526), Transaction(recipient='Jon', sender='Bob', amount=2, timestamp=1542722888.0491583)]
 
 previous hash: 00b718417a69733cd1b8a5307cce3b2c8fe3115e89f1d52e436e7ec9a1ab607c
 
 nonce: blalba]

In [28]:
chain.is_valid

True

Facinating part is that we can have any number of arbitrary rulles. As long as each helper (a.k.a miner) validates prevous work the system is stable. Each helper gets reward if her block becomes part of the chain and looses that reward if someone else decides it's wrong. Since no one has a guarantee to validate their own work everyone has to abide by acepted set of rules.

## Asymetric encryption to the rescue

There is one last major problem. I can just register a transaction from Alise (since she has a lot of help units) to Bob. It will be all valid but Alise might not even know she transfared her help units to Bob.

Sadly python doesn't have many encryption features in its standard library. Cryptography is a big field. There are lot's of libs that can generate keys like PyCryptodome but we need something simple here. [ecdsa](https://github.com/warner/python-ecdsa) is the simplest to use for signature purpuses. After `pip install ecdsa` we can import and use it.

In [29]:
import ecdsa
from base64 import b64decode, b64encode

sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
vk = sk.get_verifying_key()
signature = sk.sign(vk.to_string())
vk.verify(signature, vk.to_string())

True

The idea looks simple but `sk` (Signature Key) and `vk` (Verifying Key) are objects so we can't just put them into transaction.

In [30]:
sk, vk

(<ecdsa.keys.SigningKey at 0x7fdfb6eec0b8>,
 <ecdsa.keys.VerifyingKey at 0x7fdfb6eec278>)

There are helper methods but it still looks ugly.

In [31]:
vk.to_string()

b'\x8c\x06\xfd\x92\x8ac\xf1\x7fm@\x00\r\xfboB\t\xf1\x11]\x87+K\xf8u\xa5:\x1aAW53\xc5\xc9\x03\xbb\xeaS$\xc2Y\xb3\xb5\xae\x98-M\xef\xber \xb3\x8e\x1c\x9c5\x8e\xb3\xd3\xda\xd9\xf1~\xd1\xaa'

In [32]:
signature

b')\x88q\x07\x06e\xa7\xe8P(\xe7\xeac\xb4\xcf\xd1\xbbC\xb1\x1c\xd1\xaa\xdb\x9f\t\xc5w\x0f\xed\x9c\xb5\xf2:\xd8\x14\xc9\xfb^\xf2 \xb9\xd0q\x14a\xde\x0e{[If1\x04\x05\x8c\x19W\xf0LQe8\x9f3'

Let's encode them.

In [33]:
from base64 import b64encode, b64decode

In [34]:
public_key = b64encode(vk.to_string())
sign = b64encode(signature)

print('public key: {} \nsignature:  {}'.format(public_key, sign))

public key: b'jAb9kopj8X9tQAAN+29CCfERXYcrS/h1pToaQVc1M8XJA7vqUyTCWbO1rpgtTe++ciCzjhycNY6z09rZ8X7Rqg==' 
signature:  b'KYhxBwZlp+hQKOfqY7TP0btDsRzRqtufCcV3D+2ctfI62BTJ+17yILnQcRRh3g57W0lmMQQFjBlX8ExRZTifMw=='


That looks a little better. Maybe we can do better?

In [35]:
from binascii import hexlify, unhexlify

In [36]:
public_key = hexlify(vk.to_string())
sign = hexlify(signature)

print('public key: {} \nsignature:  {}'.format(public_key, sign))

public key: b'8c06fd928a63f17f6d40000dfb6f4209f1115d872b4bf875a53a1a41573533c5c903bbea5324c259b3b5ae982d4defbe7220b38e1c9c358eb3d3dad9f17ed1aa' 
signature:  b'298871070665a7e85028e7ea63b4cfd1bb43b11cd1aadb9f09c5770fed9cb5f23ad814c9fb5ef220b9d0711461de0e7b5b49663104058c1957f04c5165389f33'


It looks prettier but longer. I'll stick with shourter version here. Onward to our signature field!

In [37]:
# Our new transaction data
@dataclass
class Transaction:
    recipient: bytes
    sender: bytes
    amount: float
    signature: bytes = b'' # New signature filed
    timestamp: float = field(default_factory=time)

In [38]:
def sign(public: bytes, private: bytes) -> bytes:
    pk = b64decode(private)
    signing_key = ecdsa.SigningKey.from_string(pk, curve=ecdsa.SECP256k1)
    signature = b64encode(signing_key.sign(public))
    return signature


def is_valid(transaction: Transaction) -> bool:
    pub_key = ecdsa.VerifyingKey.from_string(b64decode(transaction.sender))
    return pub_key.verify(b64decode(transaction.signature), transaction.sender)


class Person:
    public: bytes
    private: bytes

    def __init__(self):
        sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
        vk = sk.get_verifying_key()
        self.public = b64encode(vk.to_string())
        self.private = b64encode(sk.to_string())

    def __repr__(self):
        return str(self.public)

    def send_help(self, to:bytes, amount: float) -> Transaction:
        return Transaction(
            recipient=to,
            sender=self.public,
            signature=sign(self.public, self.private),
            amount=amount
        )

Now if Alis wants to send help to Bob, first she and Bob have to genarate key pairs.

In [39]:
alise = Person()
bob = Person()
jon = Person()

In [40]:
alise.send_help(to=bob.public, amount=2.0)

Transaction(recipient=b'3dJw2vTbLZgmDwRgnk5JBDBNIXjk3n2XkJw4DxGTu0hmBv9M8TOAGJBjDGd5tWD2E1h4TkQMUqLzfNgLpQ/KMA==', sender=b'AEFdQJ0snX6PiXe+DQTYzeiHeDKSc5if4qzrAvoD0SkjcsXjCh5v0WLhq5rn38uEnnmb8gDXiWiWtrCWk+G9HA==', amount=2.0, signature=b'Pb2RHLslVBVMrTDVQc7tUWKwUpzENXulrCzRrA64cL4xAlyHEXAqoV11GmdLTyY7D6Hp9Nv9njrCv1ZzfoKOXA==', timestamp=1542722890.0637498)

In [41]:
chain = Chain()

In [42]:
block = Block()

In [43]:
block.append(alise.send_help(to=bob.public, amount=10.0))
block.append(bob.send_help(to=jon.public, amount=5.0))
block.append(jon.send_help(to=alise.public, amount=2.5))

In [44]:
add_block_with_proof_of_work(chain, block)

Found hash 0005ba69e84aeef48407dff1c22f68849ad7148f9410b1789213cfe099483e61 after 74 iterations


In [45]:
chain.blocks

[transactions: [Transaction(recipient='Alise', sender='Alise', amount=2.0, signature=b'', timestamp=1542722890.127797)]
 
 previous hash: coinbase
 
 nonce: 3e7ad093-3d27-4c78-becb-f43971821b97,
 transactions: [Transaction(recipient=b'3dJw2vTbLZgmDwRgnk5JBDBNIXjk3n2XkJw4DxGTu0hmBv9M8TOAGJBjDGd5tWD2E1h4TkQMUqLzfNgLpQ/KMA==', sender=b'AEFdQJ0snX6PiXe+DQTYzeiHeDKSc5if4qzrAvoD0SkjcsXjCh5v0WLhq5rn38uEnnmb8gDXiWiWtrCWk+G9HA==', amount=10.0, signature=b'c+vb0FOajdkLIUJYQ0iAImNBoqphSsc8B2rL7TzbpAy/93r0m8qeifhcqvmlGK2zdMUNeZb+Vgh1pNhL+KCEmA==', timestamp=1542722890.4596875), Transaction(recipient=b'Hm6ThkxMJl+6eLaPOcaIxYG/b9w7MivLv+xAUHDY+4D08yojyfkBmJn445lC7KjDj2mnQGqlNWQZC+L4pP4kJA==', sender=b'3dJw2vTbLZgmDwRgnk5JBDBNIXjk3n2XkJw4DxGTu0hmBv9M8TOAGJBjDGd5tWD2E1h4TkQMUqLzfNgLpQ/KMA==', amount=5.0, signature=b'c3vxAkgbquVTAS9WMsCpdiLzUqDthgqVPlqiSFfoHjTDPnVnSd3ZOt6EaRKYHfNdE5IuEcKnWRfcqTzR07uvrA==', timestamp=1542722890.5565984), Transaction(recipient=b'AEFdQJ0snX6PiXe+DQTYzeiHeDKSc5if4qzrAvoD0S

In [46]:
block = Block()

In [47]:
block.append(alise.send_help(to=bob.public, amount=2.5))
block.append(bob.send_help(to=jon.public, amount=5.0))
block.append(jon.send_help(to=alise.public, amount=10.0))

In [48]:
add_block_with_proof_of_work(chain, block)

Found hash 0043a4c7b715bd113389aee7fb9f52bf16439889ecfa013cda15036208d46c2d after 27 iterations


In [49]:
chain.blocks

[transactions: [Transaction(recipient='Alise', sender='Alise', amount=2.0, signature=b'', timestamp=1542722890.127797)]
 
 previous hash: coinbase
 
 nonce: 3e7ad093-3d27-4c78-becb-f43971821b97,
 transactions: [Transaction(recipient=b'3dJw2vTbLZgmDwRgnk5JBDBNIXjk3n2XkJw4DxGTu0hmBv9M8TOAGJBjDGd5tWD2E1h4TkQMUqLzfNgLpQ/KMA==', sender=b'AEFdQJ0snX6PiXe+DQTYzeiHeDKSc5if4qzrAvoD0SkjcsXjCh5v0WLhq5rn38uEnnmb8gDXiWiWtrCWk+G9HA==', amount=10.0, signature=b'c+vb0FOajdkLIUJYQ0iAImNBoqphSsc8B2rL7TzbpAy/93r0m8qeifhcqvmlGK2zdMUNeZb+Vgh1pNhL+KCEmA==', timestamp=1542722890.4596875), Transaction(recipient=b'Hm6ThkxMJl+6eLaPOcaIxYG/b9w7MivLv+xAUHDY+4D08yojyfkBmJn445lC7KjDj2mnQGqlNWQZC+L4pP4kJA==', sender=b'3dJw2vTbLZgmDwRgnk5JBDBNIXjk3n2XkJw4DxGTu0hmBv9M8TOAGJBjDGd5tWD2E1h4TkQMUqLzfNgLpQ/KMA==', amount=5.0, signature=b'c3vxAkgbquVTAS9WMsCpdiLzUqDthgqVPlqiSFfoHjTDPnVnSd3ZOt6EaRKYHfNdE5IuEcKnWRfcqTzR07uvrA==', timestamp=1542722890.5565984), Transaction(recipient=b'AEFdQJ0snX6PiXe+DQTYzeiHeDKSc5if4qzrAvoD0S

Okay, we sufficiently rediculus-ified our output but most of it because of all the hashing strings. If we ignore those things are actually very simple: 

1. There are transactions with who gave help to who and a signature so there is a way to validate the transaction. 
2. There is a `nonce` field that we use to calculate hash for next block. 
3. There is a prevous hash filed.

You probably noticed that anyone can send any amount of help units. How do we make sure that help units appear only from mining? We don't. We just make a rule that people can spend only as much as they have and only miners can assign value to themselfs. As long as miners verify each others work they will follow any kind of verifiable rulles. In order to make sure a person doesn't sepend more than she has miners just need to sum up all the transactions involving that public address. Rule that transactions are valid only on rainy wheather will not work becouse our chain doesn't contain this information.

## Is this all there is to blockchain and cryptocoins?

Yes, it covers main consepts. And no, I wouldn't trast more than a dollar to this particular one (even if we don't take into account lots of small missing parts). When I started this quest I suspected that its possible to uncover details that I wouldn't find just by reading articles about blockchain. The digest of what I learned while writing this simple blockchain is in `Code Hash` article.

# Code hash

I went on a quset to implement cryptocurency consepts in code. In my prevous article `In code we trust` I argued that one must impement cryptocurency consepts in code in order to understand them. I still stand by that statement but now I understood why very few blogers do that. It is very borring to read. Don't get me wrong it was fun to think about it and write code but I must be honest with myself and admit that very few people will want to read it carefully. My `Chain of thought` jupyter notebook is for those brave soles. The rest of the article focuses on lessons I learned while writing code for a simple blockchain.

[pic]

## Not money
It's better to not think of cryptocurrency as money. It's an immutable list with transactions. When you add entities that maintain the ladger and validate each outhers entries, the most apparant application is to "send" units of value from one address to another. The true value of such ladgers comes from enabaling cooperation between people, autobeings, aliens, unicorns, or anything else that follows the protocol. 

## Not complicated
It turns out that the main consepts are very simple. I reconstructed them with very few lines of code starting from a simple list of transactions and hardening it against temparing. Specific implementation matters a lot. That is what separates successful coin worth millions of dollars from worthless crap.

## Not a database
I kind of knew that before but I didn't realy appriciated how special the first and the last blocks are. Blockchain itself is a crappy database. It's only good for verifying chain integety. It's incredebly ineficient event at that task. Any blockchain should have better data format on top of it. It would be interesting to check what data formats are used by famous coins.

## Game of rulles
It's facinating that a lot of coin rules don't need to be spelled out explicitly in the blockchain. They are reinforced only by miner implementation. For example blockchain doesn't need to contain wallet balances. It's calculated by summing up all the transactions with the wallet. This is cool feature but it's easy to create ambigious rulles or rulles with uninteded consiqunces. Let's say we allow some negative balance in our transactions. It might work for a while untill some smart soule realizes that she can create wallets, empty them down to that negative amount into another wallet, and abandon the once with negative amount. Even though the example is simple relationship between negative account balance and the ability to limitlessly print the coin is not very apperant. It seems to me that complexity increases with the number of those rules. With complexity the chanse that something like that is possible increases as well.

## Validation right
Destributing validation right using prof of work is genious but it feels wrong. It reminds me the famous Arthur C. Clark story [The Nine Billion Names of God](https://en.wikipedia.org/wiki/The_Nine_Billion_Names_of_God). No wonder people are looking for alternatives. I wish there was some quantum randomnes law that could distribute validation right and didn't accept bribes. I doubt it's possible in this universe though.

## What's next?
First I'm going to look into some real blockchain fromats like Bitcon, Etherioum, Steem, and maybe another interesting coin. There must be immutable database-like structures that are alrady implemented. After that I'll work on some tools that can tell me some stats about blockchains. For example I'm very courious what's the ususal Bitcoin tail. The rule is that the longest chain wins but how many blocks die on average replaced by a longer chain.